<a href="https://colab.research.google.com/github/SugarC21/colab_Open_WebUI/blob/main/Colab_Open_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Open WebUI + Ollama in Colab**

This notebook sets up **Open WebUI** (Python 3.11) and **Ollama** on Google Colab. Both can optionally be installed in Google Drive to persist across sessions. **ngrok** is used to provide a public URL for the Open WebUI interface (port 8081). Ollama runs in the background on port 11422.

### **Features**
1. **Install** Open WebUI in a Python 3.11 virtual environment.
2. **Optionally** persist Open WebUI and Ollama in Google Drive.
3. **Expose** Open WebUI via ngrok.
4. **Hide** Ollama from the public URL (internal only).

In [ ]:
#@title **Setup**
use_gdrive = True #@param {type:"boolean"}
use_ngrok_auth = False #@param {type:"boolean"}

import os

BASE_PATH = "/content"
if use_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/Open-WebUI"
    os.makedirs(BASE_PATH, exist_ok=True)

print("Using base path:", BASE_PATH)
print("Using ngrok auth token:", use_ngrok_auth)

## (Optional) Using Authentication Token
If you want a stable subdomain or advanced ngrok features, toggle `use_ngrok_auth` and provide your token in one of the following ways:
- **Environment variable**: `%env ngrok_auth_token=YOUR_TOKEN`  
- **Colab secrets**: Go to **Runtime** > **RunTime manager** > **Secrets** > add `ngrok_auth_token`.


## Install Dependencies
Installs Python 3.11, venv, development packages, and system tools.

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y python3.11 python3.11-venv python3.11-dev pciutils lshw

## Install Ollama
If `use_gdrive` is **True**, Ollama is stored in Drive; otherwise it is installed system-wide.

In [ ]:
if use_gdrive:
    print("Persisting Ollama to Drive...")
    import os
    ollama_dir = os.path.join(BASE_PATH, 'ollama')
    ollama_bin = os.path.join(ollama_dir, 'ollama')

    if os.path.exists(ollama_bin):
        print("Ollama already exists in Drive.")
    else:
        print("Downloading Ollama (Linux x86_64) to Drive...")
        os.makedirs(ollama_dir, exist_ok=True)
        OLLAMA_VERSION = "v0.0.16"  # example pinned version
        DOWNLOAD_URL = f"https://github.com/jmorganca/ollama/releases/download/{OLLAMA_VERSION}/ollama-{OLLAMA_VERSION}-Linux-x86_64.tar.gz"
        !wget -q "$DOWNLOAD_URL" -O /tmp/ollama.tar.gz
        !tar -xzf /tmp/ollama.tar.gz -C "$ollama_dir" --strip-components 1
        !chmod +x "$ollama_bin"
        print("Ollama persisted at:", ollama_bin)
else:
    print("Installing Ollama system-wide (ephemeral)")
    !curl -fsSL https://ollama.com/install.sh | sh
    print("Ollama installed for this session.")

## Set Up Virtual Environment & Install Open WebUI

In [ ]:
import os

venv_path = os.path.join(BASE_PATH, "venv")
if not os.path.exists(venv_path):
    print("Creating Python 3.11 virtual environment...")
    !python3.11 -m venv "$venv_path"

print("Upgrading pip...")
!"$venv_path/bin/python" -m pip install --upgrade pip

print("Installing Open WebUI...")
!"$venv_path/bin/pip" install open-webui

print("Open WebUI installation complete.")

## Create a Script to Start Both Servers
Ollama runs on port **11422**. Open WebUI runs on port **8081**.

In [ ]:
import os

server_script_path = os.path.join(BASE_PATH, 'start_servers.py')
if use_gdrive:
    ollama_bin = os.path.join(BASE_PATH, 'ollama', 'ollama')
else:
    ollama_bin = 'ollama'

script_content = f'''\
import subprocess, threading, time\n\n
OLLAMA_CMD = "{ollama_bin}"\n\n
def start_ollama():\n",
    subprocess.run([OLLAMA_CMD, 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)\n\n
def start_open_webui():\n",
    subprocess.run(['./venv/bin/open-webui', 'serve', '--port', '8081'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)\n\n
threading.Thread(target=start_ollama).start()\n",
time.sleep(5)\n",
threading.Thread(target=start_open_webui).start()\n"""

with open(server_script_path, 'w') as f:
    f.write(script_content)

print("Created:", server_script_path)

## Start Servers & Expose Open WebUI via ngrok
After a brief wait, the Open WebUI URL will be displayed.

In [ ]:
!pip install pyngrok --quiet

import time, os
from pyngrok import ngrok

# Attempt to retrieve ngrok token from environment or secrets
def get_ngrok_token():
    token_env = os.environ.get('ngrok_auth_token', '')
    if token_env:
        return token_env
    # If colab secrets usage is desired, user might do something like:
    # from google.colab import runtime
    # secrets = runtime.authorized_users.secrets.get('ngrok_auth_token','')
    # This can vary with different Colab setups. Provide your own logic if needed.
    return ''

if use_ngrok_auth:
    token = get_ngrok_token()
    if token:
        ngrok.set_auth_token(token)
        print("ngrok auth token set.")
    else:
        print("No 'ngrok_auth_token' found. Proceeding without auth token.")

print("Starting servers...")
venv_path = os.path.join(BASE_PATH, 'venv')
server_script_path = os.path.join(BASE_PATH, 'start_servers.py')
!"$venv_path/bin/python" "$server_script_path" &
time.sleep(20)

webui_tunnel = ngrok.connect(8081, "http")
print("Open WebUI URL:", webui_tunnel.public_url)

# Hidden tunnel for Ollama
ollama_tunnel = ngrok.connect(11422, "http")

print("\nSetup complete.")